<a href="https://colab.research.google.com/github/onleey/Machine-Learning_demo/blob/master/653_%EC%9E%90%EC%97%B0%EC%96%B4%EC%B2%98%EB%A6%AC_MNIST_%EB%AC%B8%EC%9E%A5%EA%B0%90%EC%A0%95%EB%B6%84%EB%A5%98_CNN%EB%AA%A8%EB%8D%B8_%EC%A0%95%EB%B0%80%EB%8F%84.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
from google.colab import drive
drive.mount('/content/drive')

%cd '/content/drive/MyDrive/001'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/001


In [11]:
import tensorflow as tf
tf.__version__

'2.15.0'

In [12]:
from tensorflow import keras
from keras.datasets import mnist

In [13]:
print(dir(keras.datasets))

['__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__', 'boston_housing', 'cifar10', 'cifar100', 'fashion_mnist', 'imdb', 'mnist', 'reuters']


In [14]:
# 필요한 모듈 임포트
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Flatten, Dense

#MNIST 데이터셋 가지고 오기
(x_train, y_train), (x_test,y_test) = mnist.load_data()
x_train, x_test = x_train/ 255.0, x_test / 255.0 #데이터 정규화
#MNIST의 픽셀값 범위가 0~255이기 때문에 모든 수를 1미만으로 정규화 시키기 위해 255.0을 나누어 준다.
#print(x_train.shape) #(60000, 28, 28)
#plt.imshow(x_train[7211])

#tf.data를 사용하여 데이터셋을 섞고 배치 만들기
ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(10000)
train_size = int(len(x_train)*0.7) #학습 세트 : 검증세트 = 7:3

#batch() > 처리하려고 들고오는 데이터의 수 (괄호안에 숫자만큼 가지고와서 데이터 처리)
train_ds = ds.take(train_size).batch(20)
val_ds = ds.skip(train_size).batch(20)

########################
#MNIST 분류 모델 구성###
########################

#Sequential() : 순차적인 신경망 구성할 때 사용되는 클래스
model = Sequential()
#MNIST 이미지의 크기가 28*28픽셀로 구성되어있는데, flatten을 이용해서 1차원으로 펼쳐준다.
model.add(Flatten(input_shape=(28,28))) #input layer// 인풋층
model.add(Dense(20, activation='relu')) #hidden layer// 히든층
model.add(Dense(20, activation='relu')) #hidden layer// 히든층
model.add(Dense(10, activation='softmax')) #output layer // 아웃풋층

#############
##모델 생성##(알고리즘 생성)
#############

model.compile(loss='sparse_categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])
#model.complie(loss=categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])

#실제값과 예측값의 차이를 구하는 손실함수를 구할때 //  정해져있음 그냥 공식임// 가져다 쓰면됨
#optimizer => 경사하방법을 구하는 방법 중 하나 지정
#class(정답)가 3개 이상일때 ==> loss='sparse_categorical_crossentropy'
#class(정답)가 2개 이하일때 ==> loss=categorical_crossentropy'

#모델(알고리즘) 학습
hist = model.fit(train_ds, validation_data=val_ds, epochs=10)

#모델 평가
print('모델 평가')
model.evaluate(x_test, y_test)

#모델 정보 출력
model.summary()

#모델 저장
model.save('mnist_model.h5')

11490434/11490434 [==============================] - 0s 0us/step
Epoch 1/10
2100/2100 [==============================] - 8s 3ms/step - loss: 0.7526 - accuracy: 0.7854 - val_loss: 0.3606 - val_accuracy: 0.8946
Epoch 2/10
2100/2100 [==============================] - 5s 2ms/step - loss: 0.3425 - accuracy: 0.9015 - val_loss: 0.2932 - val_accuracy: 0.9158
Epoch 3/10
2100/2100 [==============================] - 6s 3ms/step - loss: 0.2934 - accuracy: 0.9162 - val_loss: 0.2645 - val_accuracy: 0.9227
Epoch 4/10
2100/2100 [==============================] - 7s 3ms/step - loss: 0.2652 - accuracy: 0.9252 - val_loss: 0.2384 - val_accuracy: 0.9311
Epoch 5/10
2100/2100 [==============================] - 7s 3ms/step - loss: 0.2428 - accuracy: 0.9307 - val_loss: 0.2253 - val_accuracy: 0.9354
Epoch 6/10
2100/2100 [==============================] - 10s 5ms/step - loss: 0.2282 - accuracy: 0.9338 - val_loss: 0.2201 - val_accuracy: 0.9353
Epoch 7/10
2100/2100 [==============================] - 5s 2ms/step - 

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


##문장 감정 분류 CNN 모델

In [36]:
# 필요한 모듈 임포트
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from keras import preprocessing
from keras.models import Model
from keras.layers import Input, Embedding, Dense, Dropout, Conv1D, GlobalMaxPool1D, concatenate

# 데이터 읽어오기
train_file = "./datasets/ratings3.csv"
data = pd.read_csv(train_file, delimiter=',')
features = data['Q'].tolist()
labels = data['label'].tolist()

print(data.head())
print(data['label'].value_counts()) #0:일상다반사 // 1:이별(부정) //2.사랑(긍정)

ParserError: ignored

###CNN(Convolutional neural network, 합성곱 신경망)
 - 시각적 이미지 분석 및 분류에 일반적으로 사용되는 인공신경망이다.

In [2]:
# 단어 인덱스 시퀀스 벡터
corpus = [preprocessing.text.text_to_word_sequence(text) for text in features]
print(corpus)

In [2]:
from collections import OrderedDict

tokenizer = preprocessing.text.Tokenizer()
print(dir(tokenizer))
tokenizer.fit_on_texts(corpus)

#문장내 모든 단어를 수퀀스 번호로 변환한다.
sequences = tokenizer.texts_to_sequences(corpus)
print(sequences)
word_index = tokenizer.word_index
print(word_index)

MAX_SEQ_LEN = 15 #단어 시퀀스 벡터 크기
padded_seqs = preprocessing.sequence.pad_sequences(sequences, maxlen=MAX_SEQ_LEN, padding='post')
print(padded_seqs)
print(padded_seqs[0:1])

#학습용,검증용,테스트용 데이터 셋 생성
#학습용,검즘용.테스트용 = 7:2:1

ds = tf.data.Dataset.from_tensor_slices((padded_seqs, labels))
ds = ds.shuffle(len(features))

train_size = int(len(padded_seqs)*0.7)
val_size = int(len(padded_seqs)*0.2)
test_size = int(len(padded_seqs)*0.1)

train_ds = ds.take(train_size).batch(20)
val_ds = ds.skip(train_size).take(val_size).batch(20)
test_ds= ds.skip(train_size+val_size).take(test_size).batch(20)

#하이퍼파라미터
dropout_prob = 0.5 #결과보고 과적합 비율 조절 필요
EMB_SIZE = 128
EPOCH = 35
VOCAB_SIZE = len(word_index) +1 #전체 단어 수

#CNN모델 정의
input_layer = Input(shape=(MAX_SEQ_LEN))
print('input_layer:', input_layer.shape)


#임베딩 크기 생성
embedding_layer= Embedding(VOCAB_SIZE, EMB_SIZE, input_length=MAX_SEQ_LEN)(input_layer)
print('embedding_layer:', embedding_layer.shape) #embedding_layer: (None, 15, 128)
dropout_emb = Dropout(rate=dropout_prob)(embedding_layer)
print('dropout_emb:', dropout_emb.shape)         #dropout_emb: (None, 15, 128)

#CNN tensor 사이즈 계산하기 : (이미지 크기 - kernel_size)/strides + 1
#풀링 연산이란 합성곱 연산결과로 나온 특징맵의 크기를 줄이거나 주요한 특징을 추출하기 위해서 사용되는 연산이다.
#풀링 연산에는 최대풀링과 평균풀링 연산이 있는데 주로 최대 풀링 연산을 사용한다.
#풀링 연산에도 합성곱연산에서 사용되는 윈도우 크기, 스트라이드, 패딩 개념등이 동일하게 적용된다.
conv1 = Conv1D(filters = 128, kernel_size=3, padding='valid', activation=tf.nn.relu)(dropout_emb)
print('conv1:', conv1.shape) #conv1: (None, 13, 128)
#filter를 처리하면 셀의 크기가 -2되는데, 원래 사이즈 데이터를 가지고싶다면 padding='same'을 주면된다.
pool1 = GlobalMaxPool1D()(conv1)
print('pool1:',pool1.shape) #pool1: (None, 128)
conv2 = Conv1D(filters = 128, kernel_size=4, padding='valid', activation=tf.nn.relu)(dropout_emb)
print('conv2:', conv2.shape) #conv2: (None, 12, 128)
pool2 = GlobalMaxPool1D()(conv2)
print('pool2:',pool2.shape) #pool2: (None, 128)
conv3 = Conv1D(filters = 128, kernel_size=5, padding='valid', activation=tf.nn.relu)(dropout_emb)
print('conv3:', conv3.shape) #conv3: (None, 11, 128)
pool3 = GlobalMaxPool1D()(conv3)
print('pool3:',pool3.shape) #pool3: (None, 128)


#3,4,5 - gram 이후 합치기
concat = concatenate([pool1, pool2, pool3])
print('concat:', concat.shape) #concat: (None, 384)

hidden = Dense(128, activation=tf.nn.relu)(concat)  #relu - 10보다 작으면 0을 / 그 이상이면 그대로 값 반환
dropout_hidden = Dropout(rate=dropout_prob)(hidden) #과적합(overfitting)방지를 위한 dropout
logits = Dense(3, name='logits')(dropout_hidden)
print('logits:', logits.shape) #logits : (None, 3)
predictions = Dense(3, activation=tf.nn.softmax)(logits)
print('predictions:', predictions.shape) #predictions: (None,3)

#모델생성
model = Model(inputs=input_layer, outputs=predictions)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

#모델 학습
model.fit(train_ds, validation_data=val_ds, epochs=EPOCH)

#모델 평가
loss, accuracy = model.evaluate(test_ds, verbose=1)
print('Accuracy: %f' % (accuracy * 100))
print('loss: %f'% (loss))

model.save('./source/cnn_model.h5')

** 문답 데이터  **

In [37]:
import tensorflow as tf
import pandas as pd
from tensorflow import keras
from keras.models import Model, load_model
from keras import preprocessing

# 데이터 읽어오기
train_file = "./datasets/ratings3.csv"
data = pd.read_csv(train_file, delimiter=',')
features = data['Q'].tolist()
labels = data['label'].tolist()

# 단어 인덱스 시퀀스 벡터
corpus = [preprocessing.text.text_to_word_sequence(text) for text in features]
tokenizer = preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(corpus)
sequences = tokenizer.texts_to_sequences(corpus)
MAX_SEQ_LEN = 15 # 단어 시퀀스 벡터 크기
padded_seqs = preprocessing.sequence.pad_sequences(sequences, maxlen=MAX_SEQ_LEN, padding='post')

# 테스트용 데이터셋 생성
ds = tf.data.Dataset.from_tensor_slices((padded_seqs, labels))
ds = ds.shuffle(len(features))
test_ds = ds.take(2000).batch(20) # 테스트 데이터셋

# 감정 분류 CNN 모델 불러오기
model = load_model('cnn_model.h5')
model.summary()
model.evaluate(test_ds, verbose=2)

# 테스트용 데이터셋의 10212번째 데이터 출력
print("단어 시퀀스 : ", corpus[10212])
print("단어 인덱스 시퀀스 : ", padded_seqs[10212])
print("문장 분류(정답) : ", labels[10212])

# 테스트용 데이터셋의 10212번째 데이터 감정 예측
picks = [10212]
predict = model.predict(padded_seqs[picks])
predict_class = tf.math.argmax(predict, axis=1)
print("감정 예측 점수 : ", predict)
print("감정 예측 클래스 : ", predict_class.numpy())

ParserError: ignored

In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras import preprocessing
from keras.models import Model
from keras.layers import Input, Embedding, Dense, Dropout, Conv1D, GlobalAveragePooling1D, concatenate

#데이터 읽어오기
train_file = './datasets/chatbot_data.csv'
data = pd.read_csv(train_file, delimiter=',')
features = data['Q'].tolist()
labels = data['label'].tolist()


#d단어 인덱스 시퀀스벡터
corpus = [preprocessing.text.text_to_word_sequence(text)for text in features]
print('corpus:',corpus)

#
tokenizer = preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(corpus)
sequences = tokenizer.texts_to_sequences(corpus)
word_index = tokenizer.word_index
MAX_SEQ_LEN =15
padded_seqs= preprocessing.sequence.pad_sequences(sequences, maxlen=MAX_SEQ_LEN, padding='post')

#학습용 검증용 테스트요 ㅇ데이터셋 생성
ds= tf.data.Dataset.from_tensor_slices((padded_seqs, labels))
ds = ds.shuffle(len(features))
train_size = int(len(padded_seqs)*0.7)
val_size =int(len(padded_seqs)*0.2)
test_size =int(len(padded_seqs)*0.1)
train_ds = ds.take(train_size).batch(20)
val_ds = ds.skip(train_size).take(val_size).batch(20)
test_ds = ds.skip(train_size +val_size).take(test_size).batch(20)

#하이퍼 파라미터 설정
dropout_prob = 0.5
EMB_SIZE = 128
EPOCH = 10
VOCAB_SIZE = len(word_index)+1

##CNN 모델정의
input_layer = Input(shape=(MAX_SEQ_LEN,))
print('input_layer:', input_layer.shape)

embedding_layer = Embedding(VOCAB_SIZE, EMB_SIZE, input_length=MAX_SEQ_LEN)(input_layer)
print('embedding_later: ', embedding_layer.shape)
dropout_emb = Dropout(rate =dropout_prob)(embedding_layer)
print('dropout_emb:',dropout_emb.shape)

#CNN tensor sieze 계산하기

conv1= Conv1D(filters= 128, kernel_size=3 , padding ='valid', strides=1, activation=tf.nn.relu)(dropout_emb)
print('conv1:', conv1.shape)
Pool1 = GlobalMaxPool1D() (conv1)
print('pool1:', pool1.shape)


conv2= Conv1D(filters= 128, kernel_size=3 , padding ='valid', activation=tf.nn.relu)(dropout_emb)
print('conv1:', conv2.shape)
Pool2 = GlobalMaxPool1D() (conv2)
print('pool2:', pool2.shape)

conv3= Conv1D(filters= 128, kernel_size=3 , padding ='valid', activation=tf.nn.relu)(dropout_emb)
print('conv1:', conv3.shape)
Pool3 = GlobalMaxPool1D() (conv3)
print('pool3:', pool3.shape)


# 3,4,5 - gram 이후 합치기
concat = concatenate([pool1,pool2,pool3])
print('concat:', concat.shape)

hidden = Dense(128, activation = tf.nn.relu)(concat)
print('hidden:', hidden.shape)

dropout_hidden = Dropout(rate = dropout_prob)(hidden)
logits = Dense(3,name = 'logits')(dropout_hidden)
print('logits:', logits.shape)
predictions= Dense(3, activation = tf.nn.softmax)(logits)
print('predictions:', predictions.shape)


#모델생성
#실제값과 예측값의 차이를 구하는 손실함수를 구할때 class(정답)가 3개이상이면 sparse_categorical_corossentopy를 사용하고
#class가 정답이 2개이면 categoricla_crossentoropy를 사용한다.
#optimizer은 경사하강법을 구하는 방법을 지정한다.

#model = Model(inputs =input_layer, outputs = predictions)
model.compile(optimizer ='adam',loss ='sparse_categorical_crossentropy', metrics=['accuracy'])


#모델학습
model.fit(train_ds, validation_data= val_ds, epochs=EPOCH, verbose= 1)


#모델 평가
loss, accuracy = model.evaluate(test_ds, verbose =1)
print('accuracy: %f' %(accuracy * 100))
print('loss: %f' % (loss))
#모델저장
model.save('cnn_model.h5')


In [ ]:
import tensorflow as tf
import pandas as pd
from tensorflow import keras
from keras.models import Model, load_model
from keras import preprocessing

# 데이터 읽어오기
train_file = "./datasets/chatbot_data.csv"
data = pd.read_csv(train_file, delimiter=',')
features = data['Q'].tolist()
labels = data['label'].tolist()

# 단어 인덱스 시퀀스 벡터
corpus = [preprocessing.text.text_to_word_sequence(text) for text in features]
tokenizer = preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(corpus)
sequences = tokenizer.texts_to_sequences(corpus)
MAX_SEQ_LEN = 15 # 단어 시퀀스 벡터 크기
padded_seqs = preprocessing.sequence.pad_sequences(sequences, maxlen=MAX_SEQ_LEN, padding='post')

# 테스트용 데이터셋 생성
ds = tf.data.Dataset.from_tensor_slices((padded_seqs, labels))
ds = ds.shuffle(len(features))
test_ds = ds.take(2000).batch(20) # 테스트 데이터셋

# 감정 분류 CNN 모델 불러오기
model = load_model('cnn_model.h5')
model.summary()
model.evaluate(test_ds, verbose=2)

# 테스트용 데이터셋의 10212번째 데이터 출력
print("단어 시퀀스 : ", corpus[10212])
print("단어 인덱스 시퀀스 : ", padded_seqs[10212])
print("문장 분류(정답) : ", labels[10212])

# 테스트용 데이터셋의 10212번째 데이터 감정 예측
picks = [10212]
predict = model.predict(padded_seqs[picks])
predict_class = tf.math.argmax(predict, axis=1)
print("감정 예측 점수 : ", predict)
print("감정 예측 클래스 : ", predict_class.numpy())